In [1]:
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import time

In [2]:
with open(r'graph_centrality_codes/nodes_edges_weighted.pickle', 'rb') as handle:
    B_matrix_weighted, node_coordinates_weighted = pickle.load(handle)
    
with open(r'graph_centrality_codes/distance.pickle', 'rb') as handle:
    distance_array = pickle.load(handle)  
    
#distance_array (1st col - distance (m), 2nd col - time (s), 3rd col - time in traffic (s))

In [3]:
with open(r"script_output.pickle","rb") as handle:
    origin_destination_path_list,no_connection_list,origin_destination_path_df,no_connection_df=pickle.load(handle)

In [4]:
# origin_destination_path_df

# 0: origin_id
# 1: destination_id
# 2: path
# 3: S000_adjusted
# 4: SA01_adjusted
# 5: SA02_adjusted
# 6: SA03_adjusted
# 7: SE01_adjusted
# 8: SE02_adjusted
# 9: SE03_adjusted
# 10: SI01_adjusted
# 11: SI02_adjusted
# 12: SI03_adjusted

In [5]:
print('Number of rows in df: ' + str(len(origin_destination_path_list)))

Number of rows in df: 5162831


In [6]:
# Make a copy of the origin_destination_path_list and origin_destination_path_df

origin_destination_path_list_copy = origin_destination_path_list.copy()
origin_destination_path_df_copy = origin_destination_path_df.copy()

In [13]:
# Subset of origin_destination_path_list and origin_destination_path_df

origin_destination_path_list_copy = origin_destination_path_list[0:1000]
origin_destination_path_df_copy = origin_destination_path_df[0:1000]

In [14]:
origin_destination_path_list_dummy=origin_destination_path_df_copy[2].tolist()
   
selected_columns = origin_destination_path_df_copy[[3,4,5,6,7,8,9,10,11,12]].values    

B_matrix_add=np.zeros([70983,10],dtype=float)

for i in range(len(origin_destination_path_list_dummy)):
    print(i)
    path=origin_destination_path_list_dummy[i]
    for j in range(len(path)-1):
        start_node=path[j]
        end_node=path[j+1]
        
        idx_array = np.where((B_matrix_weighted[:, 0] == start_node) & (B_matrix_weighted[:, 1] == end_node))
        if idx_array[0].size > 0:
            idx = idx_array[0][0]
        else:
            idx_array = np.where((B_matrix_weighted[:, 0] == end_node) & (B_matrix_weighted[:, 1] == start_node))
            if idx_array[0].size > 0:
                idx = idx_array[0][0]
            else:
                idx = None

        if idx is not None:
            B_matrix_add[idx,:] += selected_columns[i,:]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
len(origin_destination_path_list_dummy)

1000

In [8]:
origin_destination_path_list_dummy=origin_destination_path_df_copy[2].tolist()
   
selected_columns = origin_destination_path_df_copy[[3,4,5,6,7,8,9,10,11,12]].values    

B_matrix_add=np.zeros([70983,10],dtype=float)

for i in range(len(origin_destination_path_list_dummy)):
    print(i)
    path=origin_destination_path_list_dummy[i]
    for j in range(len(path)-1):
        start_node=path[j]
        end_node=path[j+1]
        try:
            idx = np.where((B_matrix_weighted[:, 0] == start_node) & (B_matrix_weighted[:, 1] == end_node))[0][0]
        except:
            idx = np.where((B_matrix_weighted[:, 0] == end_node) & (B_matrix_weighted[:, 1] == start_node))[0][0]
        B_matrix_add[idx,:]+=selected_columns[i,:]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
print(sum(B_matrix_add[:, 6] != 0))

1432


In [12]:
# Count how many elements there are total in the paths of the origin_destination_path_list_copy

total_elements_copy = 0

for i in range(len(origin_destination_path_list_copy)):
    total_elements_copy += len(origin_destination_path_list_copy[i][2])


# Count how many elements there are total in the paths of the origin_destination_path_list

total_elements = 0

for i in range(len(origin_destination_path_list)):
    total_elements += len(origin_destination_path_list[i][2])


print('Total elements in the paths of the origin_destination_path_list_copy: ' + str(total_elements_copy))
print('Total elements in the paths of the origin_destination_path_list: ' + str(total_elements))

Total elements in the paths of the origin_destination_path_list_copy: 42003
Total elements in the paths of the origin_destination_path_list: 575769651


In [12]:
ratio = total_elements / total_elements_copy

In [21]:
# Rename columns

origin_destination_path_df_copy.columns = ['origin_id', 'destination_id', 'path', 'S000_adjusted', 'SA01_adjusted', 'SA02_adjusted', 'SA03_adjusted', 'SE01_adjusted', 'SE02_adjusted', 'SE03_adjusted', 'SI01_adjusted', 'SI02_adjusted', 'SI03_adjusted']

In [17]:
pairlist_actual=[]

for j in range(len(origin_destination_path_list_copy)):
    path = origin_destination_path_list_copy[j][2]
    for i in range(len(path) - 1):
        pair=(min([path[i],path[i+1]]),max([path[i],path[i+1]]))
        if pair not in pairlist_actual:
            pairlist_actual.append(pair)

print(len(pairlist_actual))

2602


In [3]:
# Initialize array of 0s with 10 columns and add to B_matrix_weighted
zero_columns = np.zeros((B_matrix_weighted.shape[0], 10))
B_matrix_weighted_array = np.hstack((B_matrix_weighted, zero_columns))
B_matrix_weighted_array = B_matrix_weighted_array.astype(float)

In [4]:
B_matrix_weighted.shape

(70983, 6)

In [10]:
B_matrix_weighted_array.shape[0]

70983

In [5]:
print(B_matrix_weighted_array.shape)

(70983, 16)


In [12]:
# B_matrix_weighted_df = pd.DataFrame(B_matrix_weighted_array)
# B_matrix_weighted_df

In [23]:
# Approach 1: Indexing the original B_matrix_weighted in array form (Linear, very slow)

start=time.time()
pairlist_test_1=[]

for index, row in origin_destination_path_df_copy.iterrows():
    path = row['path']
    for i in range(len(path) - 1):
        
        idx = np.where((B_matrix_weighted_array[:, 0] == path[i]) & (B_matrix_weighted_array[:, 1] == path[i+1]))[0]

        if len(idx) > 0:
            B_matrix_weighted_array[idx[0], 6] = B_matrix_weighted_array[idx[0], 6] + row['S000_adjusted']
            B_matrix_weighted_array[idx[0], 7] = B_matrix_weighted_array[idx[0], 7] + row['SA01_adjusted']
            B_matrix_weighted_array[idx[0], 8] = B_matrix_weighted_array[idx[0], 8] + row['SA02_adjusted']
            B_matrix_weighted_array[idx[0], 9] = B_matrix_weighted_array[idx[0], 9] + row['SA03_adjusted']
            B_matrix_weighted_array[idx[0], 10] = B_matrix_weighted_array[idx[0], 10] + row['SE01_adjusted']
            B_matrix_weighted_array[idx[0], 11] = B_matrix_weighted_array[idx[0], 11] + row['SE02_adjusted']
            B_matrix_weighted_array[idx[0], 12] = B_matrix_weighted_array[idx[0], 12] + row['SE03_adjusted']
            B_matrix_weighted_array[idx[0], 13] = B_matrix_weighted_array[idx[0], 13] + row['SI01_adjusted']
            B_matrix_weighted_array[idx[0], 14] = B_matrix_weighted_array[idx[0], 14] + row['SI02_adjusted']
            B_matrix_weighted_array[idx[0], 15] = B_matrix_weighted_array[idx[0], 15] + row['SI03_adjusted']
        elif len(idx) == 0:
            idx = np.where((B_matrix_weighted_array[:, 1] == path[i]) & (B_matrix_weighted_array[:, 0] == path[i+1]))[0]
            if len(idx) > 0:
                B_matrix_weighted_array[idx[0], 6] = B_matrix_weighted_array[idx[0], 6] + row['S000_adjusted']
                B_matrix_weighted_array[idx[0], 7] = B_matrix_weighted_array[idx[0], 7] + row['SA01_adjusted']
                B_matrix_weighted_array[idx[0], 8] = B_matrix_weighted_array[idx[0], 8] + row['SA02_adjusted']
                B_matrix_weighted_array[idx[0], 9] = B_matrix_weighted_array[idx[0], 9] + row['SA03_adjusted']
                B_matrix_weighted_array[idx[0], 10] = B_matrix_weighted_array[idx[0], 10] + row['SE01_adjusted']
                B_matrix_weighted_array[idx[0], 11] = B_matrix_weighted_array[idx[0], 11] + row['SE02_adjusted']
                B_matrix_weighted_array[idx[0], 12] = B_matrix_weighted_array[idx[0], 12] + row['SE03_adjusted']
                B_matrix_weighted_array[idx[0], 13] = B_matrix_weighted_array[idx[0], 13] + row['SI01_adjusted']
                B_matrix_weighted_array[idx[0], 14] = B_matrix_weighted_array[idx[0], 14] + row['SI02_adjusted']
                B_matrix_weighted_array[idx[0], 15] = B_matrix_weighted_array[idx[0], 15] + row['SI03_adjusted']
        if idx[0]not in pairlist_test_1:
            pairlist_test_1.append(idx[0])

end=time.time()

print(end-start)
print(len(pairlist_test_1))

KeyboardInterrupt: 

In [44]:
print(len(np.unique(pairlist_test_1)))

745


In [45]:
print(sum(B_matrix_weighted_array[:, 6] != 0))

745


In [6]:
B_matrix_weighted_dict = {(row[0], row[1]): row for row in B_matrix_weighted_array}

In [9]:
len(B_matrix_weighted_dict)

70844

In [11]:
unique_rows = np.unique(B_matrix_weighted_array, axis=0)
num_unique_rows = unique_rows.shape[0]
num_total_rows = B_matrix_weighted_array.shape[0]

if num_unique_rows < num_total_rows:
    print("There are duplicate rows in B_matrix_weighted_array.")
else:
    print("There are no duplicate rows in B_matrix_weighted_array.")

There are duplicate rows in B_matrix_weighted_array.


In [13]:
unique_rows, counts = np.unique(B_matrix_weighted_array, axis=0, return_counts=True)
duplicate_rows = unique_rows[counts > 1]

print(len(duplicate_rows))

1


In [15]:
# Create a new array that only contains the first two elements of each row
first_two_columns = B_matrix_weighted_array[:, :2]

# Find duplicate rows in the new array
unique_rows, counts = np.unique(first_two_columns, axis=0, return_counts=True)
duplicate_rows = unique_rows[counts > 1]

print(len(duplicate_rows))

135


In [21]:
print(len(unique_rows))

70844


In [19]:
# put unique_rows and counts into a single dataframe

unique_rows_df = pd.DataFrame(unique_rows)
counts_df = pd.DataFrame(counts)

unique_rows_counts_df = pd.concat([unique_rows_df, counts_df], axis=1)



In [20]:
unique_rows_counts_df

,0,1,0
0,0.0,2.0,1
1,1.0,0.0,1
2,1.0,7.0,1
3,2.0,4.0,1
4,2.0,41218.0,1
...,...,...,...
70839,50118.0,18367.0,1
70840,50119.0,36041.0,1
70841,50120.0,17743.0,1
70842,50121.0,17745.0,1


In [19]:
# Approach 2: Converting B_matrix_weighted to a dictionary for faster lookups (O(1) lookups, faster)

# Convert B_matrix_weighted_array to a dictionary for faster lookups
B_matrix_weighted_dict = {(row[0], row[1]): row for row in B_matrix_weighted_array}

start = time.time()

for _, row in origin_destination_path_df_copy.iterrows():
    path = row['path']
    for i in range(len(path) - 1):
        pair = (path[i], path[i+1])
        reverse_pair = (path[i+1], path[i])

        if pair in B_matrix_weighted_dict:
            B_matrix_weighted_dict[pair][6:16] += row['S000_adjusted':'SI03_adjusted']
        elif reverse_pair in B_matrix_weighted_dict:
            B_matrix_weighted_dict[reverse_pair][6:16] += row['S000_adjusted':'SI03_adjusted']

# Convert the dictionary back to a numpy array
B_matrix_weighted_array = np.array(list(B_matrix_weighted_dict.values()))

end = time.time()

print(end - start)

74.07736325263977


In [16]:
len(B_matrix_weighted_array[B_matrix_weighted_array[:, 6] != 0])

2602

In [19]:
# Estimated time to run the entire origin_destination_path_df

test_time = (end - start) * ratio

323348.85056469927

In [22]:
print(test_time / 60 / 60 / 24)

3.742453703703704

In [33]:
# # Approach 2: Indexing the original B_matrix_weighted in dataframe form (This would be slower than the first approach but is working)

# This iterates over each row of the origin_destination_path_df dataframe and for each row, it iterates over each pair of adjacent nodes in the path column.
# For each pair of nodes, it checks if the nodes are present in the first and second columns (or reversed) of the B_matrix_weighted_df dataframe.
# If the nodes are present, it adds the S000_adjusted value to the 6th column of the B_matrix_weighted_df dataframe.

# start=time.time()

# for index, row in origin_destination_path_df.iterrows():
#     for i in range(len(row['path']) - 1):
#         if (row['path'][i] in B_matrix_weighted_df[0] and row['path'][i + 1] in B_matrix_weighted_df[1]):
#             B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1]), 6] += row['S000_adjusted']
#             # print(B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1])])
#         elif (row['path'][i+1] in B_matrix_weighted_df[0] and row['path'][i] in B_matrix_weighted_df[1]):
#             B_matrix_weighted_df.loc[(B_matrix_weighted_df[1] == row['path'][i]) & (B_matrix_weighted_df[0] == row['path'][i + 1]), 6] += row['S000_adjusted']
#             # print(B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1])])

# end=time.time()

# print(end-start)

/var/folders/6m/88dwrhnx7m3cybxwl1p0rtq40000gn/T/ipykernel_95611/2527138943.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.05555556]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  B_matrix_weighted_df.loc[(B_matrix_weighted_df[0] == row['path'][i]) & (B_matrix_weighted_df[1] == row['path'][i + 1]), 6] += row['S000_adjusted']


112.89699602127075


In [34]:
# print(len(B_matrix_weighted_df[B_matrix_weighted_df[6] != 0]))

1006


In [43]:
# B_matrix_weighted_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,31017,30686,640120,684,24,55,0.0,0,0,0,0,0,0,0,0,0
1,40651,40669,3976160,340,28,23,0.0,0,0,0,0,0,0,0,0,0
2,15202,15358,1417450,642,36,322,0.0,0,0,0,0,0,0,0,0,0
3,15276,15427,1417420,642,36,261,0.0,0,0,0,0,0,0,0,0,0
4,15358,15473,1417460,294,36,83,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70978,37860,37869,4728880,195,36,6,0.0,0,0,0,0,0,0,0,0,0
70979,37860,37862,4728870,195,36,1,0.0,0,0,0,0,0,0,0,0,0
70980,37856,37860,4728890,166,36,9,0.0,0,0,0,0,0,0,0,0,0
70981,37877,37850,309910,407,36,0,0.0,0,0,0,0,0,0,0,0,0


In [55]:
print('Number of node pairs updated in B_matrix_weighted: ' + str(len(B_matrix_weighted_array[B_matrix_weighted_array[:, 6] != 0])))

Number of node pairs updated in B_matrix_weighted: 745


In [57]:
# Convert B_matrix_weighted_df back to a numpy array

# B_matrix_weighted_updated = B_matrix_weighted_df.to_numpy()

# Export B_matrix_weighted_updated as a pickle file

with open('B_matrix_weighted_updated.pickle', 'wb') as handle:
    pickle.dump(B_matrix_weighted_array, handle, protocol=pickle.HIGHEST_PROTOCOL)